## INSTALLING THE NECESSARY DEPENDENCIES

In [ ]:
##(install and restart runtime)
#!pip install --upgrade accelerate
#!pip install transformers[sentencepiece]

In [ ]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__


from huggingface_hub import HfFolder, notebook_login

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
notebook_login()

In [ ]:
repository_id = "Geee05/altered_data_electra"

In [ ]:
dataset = pd.read_csv('/content/All_transcript_train.csv')
tdataset= pd.read_csv('/content/All_transcript_test_new.csv')

In [ ]:
#PHQ SCORE<10 : HC else DP
tr= pd.read_csv('/content/train_split_new.csv')
ts= pd.read_csv('/content/test_split_new.csv')

## DATA PREPARATION

In [ ]:
dataset.head(2)

,Unnamed: 0,File,Text,Class
0,0,302_P,just move around a little bit when you're fin...,HC
1,1,303_P,wow okay when you're finished when she's done...,HC


In [ ]:
for i in range (0,len(dataset)):
  dataset['File'][i]=dataset['File'][i][0:3]

<ipython-input-8-0bbd421a275a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['File'][i]=dataset['File'][i][0:3]


In [ ]:
tr.head(2)

,Unnamed: 0,Participant_ID,Gender,Class,PHQ_Score,PCL-C (PTSD),PTSD Severity
0,0,302,0,0,4,0,28
1,1,303,1,0,0,0,17


In [ ]:
dataset['Class'].value_counts()

HC    126
DP     37
Name: Class, dtype: int64

In [ ]:
dataset.drop(['Class'],axis=1,inplace=True)

In [ ]:
tr['Class'].value_counts()

0    113
1     50
Name: Class, dtype: int64

In [ ]:
new_class=tr['Class']
dataset['Class']=new_class
dataset['Class'].value_counts()

0    113
1     50
Name: Class, dtype: int64

In [ ]:
for i in range (0,len(tdataset)):
  tdataset['File'][i]=tdataset['File'][i][0:3]

<ipython-input-14-54e3b7f6bd36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdataset['File'][i]=tdataset['File'][i][0:3]


In [ ]:
ts.head(2)

,Unnamed: 0,Participant_ID,Gender,PHQ_Binary,PHQ_Score,PCL-C (PTSD),PTSD Severity,predict,percentage,persad,predict_sad,predict_m_45
0,0,600,female,0,5,0,23.0,0,22.222222,22.222222,0,0
1,1,602,female,1,13,1,67.0,0,28.571429,17.857143,0,0


In [ ]:
tdataset['Group'].value_counts()

HC    39
DP    17
Name: Group, dtype: int64

In [ ]:
tdataset.drop(['Group'],axis=1,inplace=True)

In [ ]:
ts['PHQ_Binary'].value_counts()

0    35
1    21
Name: PHQ_Binary, dtype: int64

In [ ]:
new_class=ts['PHQ_Binary']
tdataset['Group']=new_class
tdataset['Group'].value_counts()

0    35
1    21
Name: Group, dtype: int64

In [ ]:
dataset.rename(columns = {'Class':'labels'}, inplace = True)
tdataset.rename(columns = {'Group':'labels'}, inplace = True)

#dataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
tdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)

In [ ]:
#resample data to deal with class imbalance
from sklearn.utils import resample
df_1=dataset[dataset['labels']==1]
df_2=dataset[dataset['labels']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=113,replace=True)
df=pd.concat([df_upsampled,df_2])
df.reset_index(inplace=True)

In [ ]:
#creating Datasets
train = Dataset.from_pandas(df, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

print(train)
print(test)

Dataset({
    features: ['index', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 226
})
Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [ ]:
leng=[]
for i in range (0,len(train['Text'])):
  leng.append(len(train['Text'][i]))

print(min(leng))
print(max(leng))

1362
21064


### Function Definition:

In [ ]:
#SEGMENT TEXTS
def segment_function(example):
  seg=example['Text']
  res=[]
  length=[]
  segmented_file=[]
  n=18000

  for iter in range (0,len(seg)):
    count=0
    for i in range (0,len(seg[iter])):
      j=i*9000
      if len(seg[iter])<18000:
          part=(seg[iter])
          res.append(part)
          count=1
          break
      elif j<=len(seg[iter]):
          segmented_file.append((example['File'][iter]))
          part=(seg[iter][j:j+n])
          res.append(part)
          #print(iter)

          count=count+1
      else:
        break
    length.append(count)

  return res#,segmented_file,length

#TOKENIZE TEXTS
tokenizer = AutoTokenizer.from_pretrained('gooohjy/suicidal-electra')
def tokenize_function(example):
  return tokenizer(example["Text"], padding='max_length',max_length=512,truncation=True,add_special_tokens = True)

In [ ]:
#LABELS CORR. TO SEGMENTS
label=[]
for i in range (0,len(train)):
  if (len(train['Text'][i])) <18000:
    label.append(train['labels'][i])
  else:
    x = ((len(train['Text'][i]))//9000) + 1
    for j in range (0,x):
      label.append(train['labels'][i])



label_t=[]
for i in range (0,len(test)):
  if (len(test['Text'][i])) <18000:
    label_t.append(test['labels'][i])
  else:
    x = ((len(test['Text'][i]))//9000) + 1
    for j in range (0,x):
      label_t.append(test['labels'][i])

#### Segmenting Train Data

In [ ]:
#ip_seg,index,leng=segment_function(train)
ip_seg=segment_function(train)

In [ ]:
#preparing dataframe for segmented texts & corresponding labels
train_data=pd.DataFrame(ip_seg,columns=['Text'])
train_data['labels']=label
train_data.head(2)

,Text,labels
0,okay hi I'm Ellie thanks for coming in today ...,1
1,might have pulled something that I'm going to...,1


#### Tokenizing Train Data

In [ ]:
trainset = Dataset.from_pandas(train_data, preserve_index=False)
trainset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 236
})

In [ ]:
tokenized_train_dataset = trainset.map(tokenize_function, batched=True)
tokenized_train_dataset

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 236
})

#### Segmenting Test Data

In [ ]:
ip_seg_t=segment_function(test)

test_data=pd.DataFrame(ip_seg_t,columns=['Text'])
test_data['labels']=label_t
test_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,this is super need I like this me either at a...,1


#### Tokenizing Test Data

In [ ]:
testset = Dataset.from_pandas(test_data, preserve_index=False)
testset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 56
})

In [ ]:
tokenized_test_dataset = testset.map(tokenize_function, batched=True)
tokenized_test_dataset

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56
})

### Review

In [ ]:
#tokenizer.model_max_length =512

In [ ]:
print(len(label))
print(len(label_t))


236
56


In [ ]:
print(len(ip_seg))
print(len(ip_seg_t))


236
56


In [ ]:
type(tokenized_train_dataset)

datasets.arrow_dataset.Dataset

## TRAINING

In [ ]:
# Set dataset format using set_format() function to specify the dataset format, making it compatible with PyTorch
tokenized_train_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
tokenized_test_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.metrics import classification_report

training_args = TrainingArguments(evaluation_strategy="epoch",num_train_epochs=35,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    output_dir=repository_id,
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token())   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('gooohjy/suicidal-electra', num_labels=2) #,ignore_mismatched_sizes=True
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    #probabilities = softmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    print((classification_report(labels,predictions)))

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/content/Geee05/altered_data_electra is already a clone of https://huggingface.co/Geee05/altered_data_electra. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.016100,1.453694,0.696429,0.320000
2,1.706800,1.125873,0.678571,0.625000
3,1.121200,1.193851,0.553571,0.576271
4,0.711500,0.625859,0.714286,0.600000
5,0.407800,0.689317,0.696429,0.370370
6,0.195900,0.870591,0.642857,0.545455
7,0.120000,1.155656,0.589286,0.530612
8,0.254800,1.505374,0.625000,0.487805
9,0.004600,2.053514,0.607143,0.476190
10,0.000900,2.463797,0.625000,0.533333


<ipython-input-39-a51742d4a3cf>:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=1050, training_loss=0.25059956645371306, metrics={'train_runtime': 2429.266, 'train_samples_per_second': 3.4, 'train_steps_per_second': 0.432, 'total_flos': 2173297317273600.0, 'train_loss': 0.25059956645371306, 'epoch': 35.0})

In [ ]:
# Save our tokenizer and create a model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file logs/events.out.tfevents.1689764275.d116dbb0ca7e.2496.0:   0%|          | 1.00/33.1k [00:00<?, ?B/…

To https://huggingface.co/Geee05/altered_data_electra
   1623564..4575e66  main -> main

   1623564..4575e66  main -> main



'https://huggingface.co/Geee05/altered_data_electra/commit/4575e668a3ef06128bf47f40764f7334834c3e9c'

## VALIDATION

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Geee05//altered_data_electra")
model_electra = AutoModelForSequenceClassification.from_pretrained("Geee05//altered_data_electra")

In [ ]:
vdataset= pd.read_csv('/content/All_transcript_val_new.csv')

In [ ]:
vdataset.head(2)

,Unnamed: 0.1,Unnamed: 0,File,Text,Group
0,0,0,600_AUDIO,okay there she is coming to go ahead and Shrin...,HC
1,1,1,602_AUDIO,this is super need I like this me either at a...,DP


In [ ]:
vdataset.rename(columns = {'Group':'labels'}, inplace = True)
vdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
vdataset['labels'].value_counts()

0    39
1    17
Name: labels, dtype: int64

In [ ]:
val = Dataset.from_pandas(vdataset, preserve_index=False)
print(val)

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [ ]:
#LABELS CORR. TO SEGMENTS

label_v=[]
for i in range (0,len(val)):
  if (len(val['Text'][i])) <18000:
    label_v.append(val['labels'][i])
  else:
    x = ((len(val['Text'][i]))//9000) + 1
    for j in range (0,x):
      label_v.append(val['labels'][i])

In [ ]:
ip_seg_v=segment_function(val)

In [ ]:
print(len(label_v))
print(len(ip_seg_v))

56
56


In [ ]:
ip_seg_v=segment_function(val)

val_data=pd.DataFrame(ip_seg_v,columns=['Text'])
val_data['labels']=label_v
val_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,this is super need I like this me either at a...,1


In [ ]:
valset = Dataset.from_pandas(val_data, preserve_index=False)
valset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 56
})

In [ ]:
tokenized_val_dataset = valset.map(tokenize_function, batched=True)
tokenized_val_dataset

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56
})

In [ ]:
tokenized_val_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [ ]:
tokenized_val_dataset

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56
})

In [ ]:
tokenized_val_dataset['input_ids'].shape

torch.Size([56, 512])

In [ ]:
logits=[]
labels=[]
for i in range (0,len(tokenized_val_dataset['Text'])):

  inputs = tokenizer(tokenized_val_dataset['Text'][i],padding='max_length',max_length=512,truncation=True,add_special_tokens = True, return_tensors="pt")
  with torch.no_grad():
    outputs = model_electra(**inputs)
    logits_l = model_electra(**inputs).logits
    #print(logits)
    logits.append(logits_l)
    labels_i = logits_l.argmax().item()
  labels.append(labels_i)

  #print(labels)

    #print(outputs)
  #print(np.array(inputs['input_ids']).shape)
  #print(np.array(inputs['token_type_ids']).shape)
  #print(np.array(inputs['attention_mask']).shape)
  #print(**inputs.shape)

  #last_hidden_states = outputs.last_hidden_state
  #result.append(outputs)

In [ ]:
logits_t=[]
labels_t=[]
for i in range (0,len(tokenized_val_dataset['Text'])):

  inputs = tokenizer(tokenized_test_dataset['Text'][i],padding='max_length',max_length=512,truncation=True,add_special_tokens = True, return_tensors="pt")
  with torch.no_grad():
    outputs = model_electra(**inputs)
    logits_l = model_electra(**inputs).logits
    #print(logits)
    logits_t.append(logits_l)
    labels_i = logits_l.argmax().item()
  labels_t.append(labels_i)


In [ ]:
len(labels)

56

In [ ]:
len(logits)

56

In [ ]:
true_labels=[]
for i in range (0,len(tokenized_val_dataset['labels'])):
  true_labels.append(tokenized_val_dataset['labels'][i])


In [ ]:
true_labels_t=[]
for i in range (0,len(tokenized_test_dataset['labels'])):
  true_labels_t.append(tokenized_test_dataset['labels'][i])


In [ ]:
len(true_labels)

56

In [ ]:
#validation set
from sklearn.metrics import classification_report

print(classification_report(true_labels,labels))

              precision    recall  f1-score   support

           0       0.79      0.59      0.68        39
           1       0.41      0.65      0.50        17

    accuracy                           0.61        56
   macro avg       0.60      0.62      0.59        56
weighted avg       0.68      0.61      0.62        56



In [ ]:
#test set
from sklearn.metrics import classification_report

print(classification_report(true_labels_t,labels_t))


              precision    recall  f1-score   support

           0       0.79      0.66      0.72        35
           1       0.56      0.71      0.63        21

    accuracy                           0.68        56
   macro avg       0.67      0.69      0.67        56
weighted avg       0.70      0.68      0.68        56

